In [1]:
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
sys.path.append('..')
import ROOT
from Tools import syncer 
from Tools import user
from Tools import helpers
import itertools
from PIL import Image
import glob

Welcome to JupyROOT 6.24/06


In [2]:
class NN:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination: self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [64, 64, 64, 64]):
        '''
        Creates the Neural Network Architecture
        '''
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables). Returns the output of the NNs of dimensions: (noutput,ndatapoints)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        
        output=torch.zeros((noutputs,ndatapoints))
        for i in range(noutputs):
            x=self.n_hat[self.combination_list[i]](features)
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=torch.flatten(x)            
        return output

    def predict_r_hat2(self, predictions,eft):
        return torch.add( 
        torch.sum( torch.stack( [(1. + predictions[(c,)]*eft[c])**2 for c in coefficients ]), dim=0),
        torch.sum( torch.stack( [torch.sum( torch.stack( [ predictions[(c_1,c_2)]*eft[c_2] for c_2 in coefficients[i_c_1:] ]), dim=0)**2 for i_c_1, c_1 in enumerate(coefficients) ] ), dim=0 ) )   

    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [3]:
def make_weight_ratio(weights, **kwargs):
    eft      = kwargs
    result = torch.ones(len(weights[()])) 
    for combination in combinations:
        if len(combination)==1:
            result += eft[combination[0]]*weights[combination]/weights[()]
        elif len(combination)==2:# add up without the factor 1/2 because off diagonals are only summed in upper triangle 
            result += (0.5 if len(set(combination))==1 else 1.)*eft[combination[0]]*eft[combination[1]]*weights[combination]/weights[()]
    return result

In [4]:
# loss functional
def f_loss(r_NN, features, predictions, base_points):
    loss = -0.5*weights[()].sum()
    for i_base_point, base_point in enumerate(base_points):
        fhat  = 1./(1. + r_NN.predict_r_hat2(predictions, base_point))
        loss += ( torch.tensor(weights[()])*( -0.25 + base_point_weight_ratios[i_base_point]*fhat**2 + (1-fhat)**2 ) ).sum()
    return loss

In [5]:
plot_directory="14_7_2022"
nEvents=30000
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 3000
plot_every    = 100



In [ ]:
# training data
import ZH_Nakamura

ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

nfeatures = len(features[0]) 
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())

train_sel, test_sel = list(map(list,torch.utils.data.random_split(range(len(features)), [len(features)//2,len(features)-len(features)//2] )))



#pT=features[:,feature_names.index('pT')]

coefficients   = ['cHW']
combinations   =  [ (), ('cHW',), ('cHW', 'cHW')]
combinations2   =  [ (), ('cHW',), ('cHW', 'cHW')]
base_points = [ {'cHW':value} for value in [-1.5, -.8, -.4, -.2, .2, .4, .8, 1.5] ]

#coefficients   = ['cHWtil']
#combinations   =  [ (), ('cHWtil',), ('cHWtil', 'cHWtil')]
#combinations2   =  [ (), ('cHWtil',), ('cHWtil', 'cHWtil')]
#base_points = [ {'cHWtil':value} for value in [-1.5, -.8, -.4, -.2, .2, .4, .8, 1.5] ]

#coefficients   =  ( 'cHW', 'cHWtil') 
#combinations   =  [(), ('cHW',), ('cHWtil',), ('cHW','cHW'), ('cHW','cHWtil'),('cHWtil','cHWtil')]
#combinations2 = [(), ('cHW',), ('cHWtil',), ('cHW','cHW'), ('cHW','cHWtil'),('cHWtil','cHW'),('cHWtil','cHWtil')]
#base_points = [ {'cHW':value1, 'cHWtil':value2} for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5]]

rows, columns = np.triu_indices(len(coefficients)+1)
w=np.zeros((len(coefficients)+1,len(coefficients)+1,features.shape[0]))

for i in range(0,len(combinations)):
    w[rows[i],columns[i],:]=torch.from_numpy(weights[combinations[i]]).float().to(device)
    if rows[i]!=columns[i]:
        w[columns[i],rows[i],:]=w[rows[i],columns[i],:]

features_train = torch.from_numpy(features[train_sel]).float().to(device)
features_test = torch.from_numpy(features[test_sel]).float().to(device)


Requested 30000 events. Simulated 30000 events and 30000 survive pT_min cut of 0.


In [7]:
coefficients = tuple(filter( lambda coeff: coeff in coefficients, list(coefficients))) 
combinations = tuple(filter( lambda comb: all([c in coefficients for c in comb]), combinations)) 

base_points    = list(map( lambda b:ZH_Nakamura.make_eft(**b), base_points ))

In [8]:
r_NN=NN(nfeatures,coefficients)

In [9]:
base_point_weight_ratios = list( map( lambda base_point: make_weight_ratio( weights, **base_point ), base_points ) )

In [10]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(sum([list(model.parameters()) for model in r_NN.n_hat.values()],[]), lr=learning_rate)

losses = []

tex = ROOT.TLatex()
tex.SetNDC()
tex.SetTextSize(0.04)
hist_colors=['b','g', 'r', 'c', 'm','y']

In [11]:
for nn in r_NN.n_hat.values():
    nn.train()

for epoch in range(n_epoch):
    #print("epoch: ", epoch)
    # Forward pass: compute predicted y by passing x to the model.
    predictions = {combination:r_NN.n_hat[combination](features).squeeze() for combination in combinations}
    
    #rows, columns = np.triu_indices(len(coefficients)+1)
    w_predicted=np.zeros((len(coefficients)+1, len(coefficients)+1, features.shape[0]))      
    for i in range(1,len(combinations)):
        w_predicted[rows[i],columns[i],:]=predictions[combinations[i]].squeeze().cpu().detach().numpy()
        if rows[i]!=columns[i]:
            w_predicted[columns[i],rows[i],:]=w_predicted[rows[i],columns[i],:]
    
    wp=np.zeros((len(coefficients),features.shape[0]))
    for i in range(0,len(coefficients)):
        wp[i,:]=2*np.sum(w_predicted[:,i+1,:],0)
    
    wpp=np.zeros((len(coefficients),len(coefficients),len(coefficients),features.shape[0]))
    for i in range(0,len(coefficients)):
        for l in range(0,len(coefficients)):
            for k in range(0,len(coefficients)):
                wpp[i,l,k,:]=2*w_predicted[i,l+1,:]*w_predicted[i,k+1,:]
    
    wpp=np.sum(wpp,0)
               
    # Compute and print loss.
    loss = f_loss(r_NN,features, predictions,base_points)
    losses.append(loss.item())
    if epoch % 100 == 99:
        print("epoch", epoch, "loss",  loss.item())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    
    

    if (epoch % plot_every)==0:
        with torch.no_grad():
                
            #print("epoch", epoch, "loss inside loop",  loss_train.item())
            #print("epoch", epoch, "test loss",  loss_test.item())
            
            for var in ['pT']:
                binning     = plot_options[var]['binning']
                np_binning= np.linspace(binning[1], binning[2], 1+binning[0])
                
                # Linear Terms
                
                

                for i in range(1,len(coefficients)+1):
                    hist_truth_0, bins  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w[0,0,:])
                    plt.step(bins[1:], hist_truth_0, label='yield', linestyle=('solid'), color='k')
                    hist_truth, bins = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w[0,i,:])
                    #plt.step(bins[1:], hist_truth/hist_truth_0, label='truth', linestyle=('dashed'), color=hist_colors[i])
                    plt.step(bins[1:], hist_truth, label='truth', linestyle=('dashed'), color=hist_colors[i])
                    hist_predicted, bins = np.histogram(features[:,feature_names.index(var)], bins, weights=w[0,0,:]*wp[i-1,:])
                    #plt.step(bins[1:], hist_predicted/hist_truth_0, label='truth', linestyle=('solid'), color=hist_colors[i])
                    plt.step(bins[1:], hist_predicted, label='truth', linestyle=('solid'), color=hist_colors[i])
                    plt.legend(['Yield ', 'Truth '+ coefficients[i-1], 'Predicted '+ coefficients[i-1]])
                    plt.savefig(os.path.join(plot_directory, "_epoch_%05i_%s__"%(epoch, var)+coefficients[i-1] + ".png" ))
                    plt.close()
                
                
                a = np.arange(len(coefficients)**2).reshape(len(coefficients),len(coefficients))
                for l in range(0,len(coefficients)):
                    for k in range(0,len(coefficients)):
                        label=combinations2[a[l,k]+len(coefficients)+1][0]+ '_' + combinations2[a[l,k]+len(coefficients)+1][1]
                        hist_truth_0, bins  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w[0,0,:])
                        plt.step(bins[1:], hist_truth_0, label='yield', linestyle=('solid'), color='k')
                        hist_truth, bins  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w[l+1,k+1,:])
                        #plt.step(bins[1:],hist_truth/hist_truth_0, label='truth', linestyle=('dashed'), color=hist_colors[a[l,k]+len(coefficients)])
                        plt.step(bins[1:],hist_truth, label='truth', linestyle=('dashed'), color=hist_colors[a[l,k]+len(coefficients)])
                        hist_predicted, bins  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w[0,0,:]*wpp[l,k,:])
                        #plt.step(bins[1:],hist_predicted/hist_truth_0, label='truth', linestyle=('solid'), color=hist_colors[a[l,k]+len(coefficients)])
                        plt.step(bins[1:],hist_predicted, label='truth', linestyle=('solid'), color=hist_colors[a[l,k]+len(coefficients)])
                        plt.legend(['Yield ' + label, 'Truth '+ label, 'Predicted '+ label])
                        plt.savefig(os.path.join(plot_directory, "_epoch_%05i_%s_"%(epoch, var)+label+".png"))
                        plt.close()
                    
                
               

epoch 99 loss 0.25761127143509793
epoch 199 loss 0.2557632368091234
epoch 299 loss 0.25390197213709187
epoch 399 loss 0.24935207543335114
epoch 499 loss 0.24555656821154942
epoch 599 loss 0.24275909454758793
epoch 699 loss 0.24241562454684434
epoch 799 loss 0.24218488165866983
epoch 899 loss 0.24197569628651153
epoch 999 loss 0.24090291757862575
epoch 1099 loss 0.2405327751277187
epoch 1199 loss 0.24028053818360157
epoch 1299 loss 0.24014786219916062
epoch 1399 loss 0.2399295174100702
epoch 1499 loss 0.2397639316393787
epoch 1599 loss 0.2395926073873346
epoch 1699 loss 0.2394565622676511
epoch 1799 loss 0.2393653526781297
epoch 1899 loss 0.23928697536415522
epoch 1999 loss 0.23921347612270247
epoch 2099 loss 0.23910134199828587
epoch 2199 loss 0.23900989816486312
epoch 2299 loss 0.23896682138171815
epoch 2399 loss 0.23894398226703822
epoch 2499 loss 0.2388625186301182
epoch 2599 loss 0.23882801341755538
epoch 2699 loss 0.23879143672114395
epoch 2799 loss 0.23876581615108275
epoch 2899 

In [12]:
for i in range(1,len(coefficients)+1):
    pTFiles=[]
    for file in os.listdir(os.getcwd()+'/'+ plot_directory):
        # check only text files
        string="__" + coefficients[i-1] + ".png"
        if file.endswith(string):
            pTFiles.append(file)
            #print(file)
    frames=[]
    for image in pTFiles:
        new_frame = Image.open(os.getcwd()+'/'+ plot_directory +'/'+image)
        frames.append(new_frame)

    frames[0].save(plot_directory + '__'+ coefficients[i-1]+'.gif', format='GIF',
                   append_images=frames[1:],
                   save_all=True,
                   duration=200, loop=0)

In [13]:
for l in range(0,len(coefficients)):
    for k in range(0,len(coefficients)):
        label=combinations2[a[l,k]+len(coefficients)+1][0]+ '_' + combinations2[a[l,k]+len(coefficients)+1][1]
        pTFiles=[]
        for file in os.listdir(os.getcwd()+'/'+ plot_directory):
            # check only text files
            string="_"+label+".png"
            if file.endswith(string):
                pTFiles.append(file)
                #print(file)
        frames=[]
        for image in pTFiles:
            new_frame = Image.open(os.getcwd()+'/'+ plot_directory +'/'+image)
            frames.append(new_frame)

        frames[0].save(plot_directory + '__'+ label+'.gif', format='GIF',
                       append_images=frames[1:],
                       save_all=True,
                       duration=200, loop=0)